# 03 — Results & Plots
## Compression-Aware Video Deepfake Detection

Generate paper-ready plots from experiment CSVs.

**Run on:** Colab, Kaggle, or locally on Mac

In [ ]:
# ── If on Colab, mount Drive and cd into repo ──
# from google.colab import drive
# drive.mount('/content/drive')
# %cd compression_aware_deepfake

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.makedirs('results/plots', exist_ok=True)
os.makedirs('results/csv', exist_ok=True)

## Load Experiment Results

In [ ]:
# Load compression evaluation CSV
comp_csv = 'results/csv/compression_eval.csv'
if os.path.exists(comp_csv):
    df_comp = pd.read_csv(comp_csv)
    print('Compression evaluation results:')
    display(df_comp)
else:
    print(f'File not found: {comp_csv}')
    print('Run training experiments first (notebook 02).')

In [ ]:
# Load ablation summary CSV
abl_csv = 'results/csv/ablation_summary.csv'
if os.path.exists(abl_csv):
    df_abl = pd.read_csv(abl_csv)
    print('Ablation summary:')
    display(df_abl)
else:
    print(f'File not found: {abl_csv}')

## Generate All Plots

In [ ]:
# Generate all paper-ready plots using the plotting script
!python scripts/plot_results.py \
    --results_dir results/csv \
    --output_dir results/plots

In [ ]:
# Display generated plots
import glob

plot_files = sorted(glob.glob('results/plots/*.png'))
for pf in plot_files:
    print(f'\n--- {os.path.basename(pf)} ---')
    img = plt.imread(pf)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(img)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

## Training Curves

In [ ]:
# Plot training logs if available
import glob

log_files = sorted(glob.glob('results/csv/train_log_*.csv'))
if log_files:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    for lf in log_files:
        name = os.path.basename(lf).replace('train_log_', '').replace('.csv', '')
        df_log = pd.read_csv(lf)
        axes[0].plot(df_log['epoch'], df_log['val_loss'], label=name)
        axes[1].plot(df_log['epoch'], df_log['val_auc'], label=name, marker='o', markersize=3)
    
    axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Val Loss')
    axes[0].set_title('Validation Loss'); axes[0].legend()
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Val AUC')
    axes[1].set_title('Validation AUC'); axes[1].legend()
    plt.tight_layout()
    plt.savefig('results/plots/training_curves.png', dpi=150)
    plt.show()
else:
    print('No training logs found.')

## Summary Table for Paper

In [ ]:
# Generate LaTeX table from results
if os.path.exists(abl_csv):
    df = pd.read_csv(abl_csv)
    pivot = df.pivot_table(values='auc', index='mode', columns='compression')
    print('\nPivot table (AUC):')
    display(pivot)
    print('\nLaTeX table:')
    print(pivot.to_latex(float_format='%.4f'))